In [2]:
import sys
sys.path.append('../')
import pickle
with open('../log/exp.pkl', 'rb') as f:
    exp = pickle.load(f)


In [3]:

keys = list(exp.keys())
print(keys[0])
print(len(exp[keys[0]]))
print(exp[keys[0]][0][2].shape)

cn,ct,k,lt,mc,mi,mk,ml,t
169
torch.Size([40, 40])


In [20]:
import torch
from torch.utils.data import Dataset, DataLoader, Sampler
import pickle
from leon_experience import Experience
import random

class BucketDataset(Dataset):
    def __init__(self, buckets, keys=None):
        buckets = {key: value for key, value in buckets.items() if value}
        self.buckets_dict = buckets
        
        self.buckets = list(buckets.values())
        # Flatten buckets
        self.buckets_item = [item for bucket in self.buckets_dict.values() for item in bucket]
        # print(self.buckets_item)
        self.keys = keys
        # filter buckets with in keys
        if self.keys is not None:
            self.buckets = [bucket for bucket in self.buckets if bucket[0][0] in keys]

    def __len__(self):
        return sum(len(bucket) for bucket in self.buckets)

    def __getitem__(self, idx):
        # print(idx)
        node, b, c = self.buckets_item[idx]
        # print(a)
        # print(b.shape)
        item = {'join_tables': node.info['join_tables'], \
                'plan_encode': b, \
                'att_encode': c, \
                'latency': node.info['latency'], \
                'cost': node.cost}
        return item

class BucketBatchSampler(Sampler):
    def __init__(self, buckets, batch_size):
        self.buckets = buckets
        self.bucket_indices = list(range(len(buckets)))
        self.batch_size = batch_size

    def __iter__(self):
        random.shuffle(self.bucket_indices)
        for i in range(0, len(self.bucket_indices), self.batch_size):
            yield [item for bucket_idx in self.bucket_indices[i:i+self.batch_size] for item in range(sum(len(bucket) for bucket in self.buckets[:bucket_idx]), sum(len(bucket) for bucket in self.buckets[:bucket_idx+1]))]
            
    def __len__(self):
        return len(self.bucket_indices) // self.batch_size
    
buckets = exp
keys = None

# 创建数据集
dataset = BucketDataset(buckets, keys)
print("all eqset:", len(dataset.buckets))
# 创建BucketBatchSampler
batch_sampler = BucketBatchSampler(dataset.buckets, batch_size=2)  # 这里的batch_size是桶的数量，每个批次包含2个桶的数据

# 创建数据加载器
dataloader = DataLoader(dataset, batch_sampler=batch_sampler)

for batch in dataloader:
    # print(batch['node'])
    print(batch['plan_encode'][0].shape)
    



all eqset: 82
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])
torch.Size([1, 720])


In [39]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import lightning.pytorch as pl
import lightning.pytorch.callbacks as plc
import torch.nn.functional as F
import torch.nn as nn
import os
import sys
from util import postgres
from util import pg_executor
from util import postgres
from util import envs
from util import plans_lib
import pytorch_lightning.loggers as pl_loggers
import pickle
import math
import json
from test_case import SeqFormer
from test_case import get_plan_encoding, configs, load_json, get_op_name_to_one_hot, plan_parameters, add_numerical_scalers
from leon_experience import Experience
import numpy as np
import ray
import time
from argparse import ArgumentParser
import copy
import wandb
import time
import random
DEVICE = 'cuda:2' if torch.cuda.is_available() else 'cpu'
# DEVICE = 'cpu'
Transformer_model = SeqFormer(
                        input_dim=configs['node_length'],
                        hidden_dim=128,
                        output_dim=1,
                        mlp_activation="ReLU",
                        transformer_activation="gelu",
                        mlp_dropout=0.3,
                        transformer_dropout=0.2,
                    )

/data1/wyz/miniconda3/envs/leon/lib/python3.8/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [56]:
class PL_Leon(pl.LightningModule):
    def __init__(self, model, optimizer_state_dict=None, learning_rate=0.001):
        super(PL_Leon, self).__init__()
        self.model = model
        self.optimizer_state_dict = optimizer_state_dict
        self.learning_rate = 0.001

    def forward(self, plans, attns):
        return self.model(plans, attns)[:, 0]

    # def make_pairs(self, join_tables, calibrations: torch.tensor, costs, latency):
    #     pairs = []
    #     calibrations1 = []
    #     calibrations2 = []
    #     costs1 = []
    #     costs2 = []
    #     labels = []
    #     for i in range(len(join_tables)):
    #         for j in range(i + 1, len(join_tables)):
    #             if join_tables[i] != join_tables[j]:
    #                 continue
    #             if latency[i] == latency[j]:
    #                 continue
    #             if latency[0] > latency[1]:
    #                 label = 0
    #             else:
    #                 label = 1
    #             labels.append(label)
    #             costs1.append(costs[i])
    #             costs2.append(costs[j])
    #             calibrations1.append(calibrations[i])
    #             calibrations2.append(calibrations[j])
        
        
    #     # make tensor
    #     labels = torch.tensor(labels, device=self.device)
    #     costs1 = torch.tensor(costs1, device=self.device)
    #     costs2 = torch.tensor(costs2, device=self.device)
        
    #     try:
    #         calibrations1 = torch.stack(calibrations1)
    #         calibrations2 = torch.stack(calibrations2)
    #     except:
    #         calibrations1 = None
    #         calibrations2 = None
    #     return labels, costs1, costs2, calibrations1, calibrations2
    def make_pairs(self, join_tables, calibrations: torch.tensor, costs, latency):
        # Find the boundaries where the join_table value changes
        boundaries = [0] + [i for i in range(1, len(join_tables)) if join_tables[i] != join_tables[i-1]] + [len(join_tables)]

        labels = []
        costs1 = []
        costs2 = []
        calibrations1 = []
        calibrations2 = []

        for i in range(len(boundaries) - 1):
            # Now join_tables[boundaries[i]:boundaries[i+1]] contains all indices for the same join_table value
            for j in range(boundaries[i], boundaries[i+1]):
                for k in range(j + 1, boundaries[i+1]):
                    if latency[j] == latency[k]:
                        continue
                    if latency[j] > latency[k]:
                        label = 0
                    else:
                        label = 1
                    labels.append(label)
                    costs1.append(costs[j])
                    costs2.append(costs[k])
                    calibrations1.append(calibrations[j])
                    calibrations2.append(calibrations[k])

        # make tensor
        labels = torch.tensor(labels, device=self.device)
        costs1 = torch.tensor(costs1, device=self.device)
        costs2 = torch.tensor(costs2, device=self.device)

        try:
            calibrations1 = torch.stack(calibrations1)
            calibrations2 = torch.stack(calibrations2)
        except:
            calibrations1 = None
            calibrations2 = None
        return labels, costs1, costs2, calibrations1, calibrations2

    def get_pair_wise_loss(self, join_tables, calibration, costs, latency):

        labels, costs1, costs2, calibrations1, calibrations2 = self.make_pairs(join_tables, calibration, costs, latency)
        if calibrations1 is None or calibrations2 is None:
            return None, None
        loss_fn = nn.BCELoss()
        calied_cost_1 = torch.log(costs1) * calibrations1
        calied_cost_2 = torch.log(costs2) * calibrations2

        sigmoid = F.sigmoid(-(calied_cost_1 - calied_cost_2))
        loss = loss_fn(sigmoid, labels.double())
    
        with torch.no_grad():
            prediction = torch.round(sigmoid)
            accuracy = torch.sum(prediction == labels).item() / len(labels)
        return loss, accuracy

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=0.001)
        if self.optimizer_state_dict is not None:
            # Checks the params are the same.
            # 'params': [139581476513104, ...]
            curr = optimizer.state_dict()['param_groups'][0]['params']
            prev = self.optimizer_state_dict['param_groups'][0]['params']
            assert curr == prev, (curr, prev)
            # print('Loading last iter\'s optimizer state.')
            # Prev optimizer state's LR may be stale.
            optimizer.load_state_dict(self.optimizer_state_dict)
            for param_group in optimizer.param_groups:
                param_group['lr'] = self.learning_rate
            assert optimizer.state_dict(
            )['param_groups'][0]['lr'] == self.learning_rate
            # print('LR', self.learning_rate)
        return optimizer

    def training_step(self, batch, batch_idx):
        plans = batch['plan_encode']
        attns = batch['att_encode']
        costs = batch['cost']
        labels = batch['latency']
        join_tables = batch['join_tables']
        
        # plans = torch.cat(plans, dim=0)
        calibrations = self(plans, attns)
        
        loss, accuracy = self.get_pair_wise_loss(join_tables, calibrations, costs, labels)
        
        if loss:
            self.log('train_loss', loss)
            self.log('train_acc', accuracy)
        return loss

In [57]:
def load_model(prev_optimizer_state_dict=None):
    model = Transformer_model.to(DEVICE)
    model = PL_Leon(model, prev_optimizer_state_dict)
    return model

def load_callbacks(logger):
    callbacks = []
    callbacks.append(plc.EarlyStopping(
        monitor='v_acc',
        mode='max',
        patience=3,
        min_delta=0.001
    ))
    if logger:
        callbacks.append(plc.ModelCheckpoint(
            dirpath= logger.experiment.dir,
            monitor='val_scan',
            filename='best-{epoch:02d}-{val_scan:.3f}',
            save_top_k=1,
            mode='min',
            save_last=False
        ))
    return callbacks

current_directory = os.getcwd()
# 获取上一级目录
parent_directory = os.path.dirname(current_directory)
logger =  pl_loggers.WandbLogger(save_dir=parent_directory + '/logs', name="exp_pair", project='leon3')

trainer = pl.Trainer(accelerator="gpu",
                        devices=[2],
                        max_epochs=100,
                        logger=logger)

prev_optimizer_state_dict = None
model = load_model().to(DEVICE)
callbacks = load_callbacks(logger=None)
# dataloader = DataLoader(dataset, batch_sampler=batch_sampler)
dataloader_train = DataLoader(dataset, num_workers=0, batch_sampler=batch_sampler)
# dataloader_val = DataLoader(dataloader, batch_size=2, shuffle=False, num_workers=0)
trainer.fit(model, dataloader_train)
prev_optimizer_state_dict = trainer.optimizers[0].state_dict()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name  | Type      | Params
------------------------------------
0 | model | SeqFormer | 16.9 K
------------------------------------
16.9 K    Trainable params
0         Non-trainable params
16.9 K    Total params
0.068     Total estimated model params size (MB)


Epoch 53:  54%|█████▎    | 22/41 [01:05<00:56,  0.34it/s, v_num=zyfb]

In [59]:
with open("../log/exp.pkl", 'wb') as f:
            pickle.dump(exp, f) 